# Text classification of tweets:

In [1]:
from __future__ import print_function

from helpersProject2 import *
from methodsProject2 import *

import numpy as np
import csv
import random

import gensim
from gensim.models import Word2Vec, Doc2Vec, FastText, KeyedVectors
import gensim.parsing.preprocessing as prep

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_validate
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import datasets, svm
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Dropout, Flatten
from keras.models import Model, load_model, Sequential
from keras.layers.embeddings import Embedding

C:\Users\Magnu\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Create word embedding based on full set:

In [4]:
features = 100
epoch = 10

#all_tweets_full, y_full = process_set('train_neg_full.txt', 'train_pos_full.txt')
print('let the word embedding commence')
#model_tot_100 = createWordEmbedding(all_tweets_full, features, epoch) #word embedding
#model_tot_100.save('model_tot_100.word2vec')
print('word embedding created')
model_tot = Word2Vec.load("model_tot_100.word2vec")

KeyboardInterrupt: 

## Cross validation for word2vec + LR and tfidf + LR:

In [ ]:
testd_tweets = open_file("test_data.txt")
all_tweets, y = process_set('train_neg.txt', 'train_pos.txt')

LR_with_w2v(all_tweets, model_tot, features, testd_tweets, y)

tfidf(testd_tweets) #with full set

# CNN

In [ ]:
list_all_tweets, y_full = easyProcess_set('train_neg_full.txt', 'train_pos_full.txt')
SEED = 42
x_train, x_validation, y_train, y_validation = train_test_split(
    list_all_tweets, y_full, test_size=.1, random_state=SEED)

model_tot_k = KeyedVectors.load('model_tot_100.word2vec')

embeddings_index = {}
for w in model_tot_k.wv.vocab.keys():
    embeddings_index[w] = model_tot_k.wv[w]
print('Found %s word vectors.' % len(embeddings_index))

num_words = 100000
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

#Finds longest tweet:
length = []
for x in x_train:
    length.append(len(x.split()))
length = max(length)+2

x_train_seq = pad_sequences(sequences, maxlen=length)

sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=length)

embedding_matrix = np.zeros((num_words, features))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
hFeatures = int(features * 0.5)

In [ ]:
tweet_input = Input(shape=(length,), dtype='int32')

tweet_encoder = Embedding(num_words, features, weights=[embedding_matrix], input_length=length, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=hFeatures, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=hFeatures, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=hFeatures, kernel_size=6, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [ ]:
filepath = "CNN128_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(
    x_train_seq,
    y_train,
    batch_size=512,
    epochs=4,
    validation_data=(x_val_seq, y_validation),
    callbacks=[checkpoint])

In [ ]:
model.evaluate(x=x_val_seq, y=y_validation)

In [ ]:
loaded_CNN_model = load_model('CNN_best_weights.01-0.8494.hdf5')
loaded_CNN_model = Model(inputs=[tweet_input], outputs=[output])
loaded_CNN_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
loaded_CNN_model.summary()
checkpoint = ModelCheckpoint(
    filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

loaded_CNN_model.fit(
    x_train_seq,
    y_train,
    batch_size=512,
    epochs=4,
    validation_data=(x_val_seq, y_validation),
    callbacks=[checkpoint])

In [ ]:
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

### Predict test set

In [ ]:
testd_tweets = easyProcess(testd_tweets)

sequences_test = tokenizer.texts_to_sequences(testd_tweets)
x_test_seq = pad_sequences(sequences_test, maxlen=length)
ycnn02 = loaded_CNN_model.predict(x=x_test_seq)

ycnn02_rounded = (np.around(ycnn02)).flatten()
ycnn02_rounded[ycnn02_rounded == 0] = -1
save_csv('test_resultCNN.csv', ycnn02_rounded)
print(ycnn02)
print(ycnn02_rounded)